### Generating a sine wave

In [1]:
import numpy as np
import scipy.io.wavfile as wav

def main():
    sample_rate_samples_per_second = 44100
    waveform_freq_hz = 440
    time_sec = 3
    waveform_type = np.sin
    wavetable_length = 64
    wave_table = np.zeros((wavetable_length,))

    for n in range(wavetable_length):
        wave_table[n] = waveform_type(2*np.pi*n / wavetable_length)

    output = np.zeros((time_sec * sample_rate_samples_per_second,))

    index = 0
    index_increment = waveform_freq_hz * wavetable_length / sample_rate_samples_per_second

    for n in range(output.shape[0]):
        output[n] = wave_table[int(np.floor(index))]
        index += index_increment
        index %= wavetable_length

    wav.write("sine440Hz.wav", sample_rate_samples_per_second, output.astype(np.float32))

if __name__ == "__main__":
    main()

### Generating a scaled sine wave (loudness adjusted)

In [2]:
import numpy as np
import scipy.io.wavfile as wav

def main():
    sample_rate_samples_per_second = 44100
    waveform_freq_hz = 440
    time_sec = 3
    waveform_type = np.sin
    wavetable_length = 64
    wave_table = np.zeros((wavetable_length,))

    for n in range(wavetable_length):
        wave_table[n] = waveform_type(2*np.pi*n / wavetable_length)

    output = np.zeros((time_sec * sample_rate_samples_per_second,))

    index = 0
    index_increment = waveform_freq_hz * wavetable_length / sample_rate_samples_per_second

    for n in range(output.shape[0]):
        output[n] = wave_table[int(np.floor(index))]
        index += index_increment
        index %= wavetable_length

    gain = -20
    amplitude = 10 ** (gain / 20)
    output *= amplitude

    wav.write("sine440Hz_scaled.wav", sample_rate_samples_per_second, output.astype(np.float32))

if __name__ == "__main__":
    main()

### Linear Interpolation to reduce ringing in the generated sound

In [ ]:
import numpy as np
import scipy.io.wavfile as wav

def interpolate_linearly(wave_table, index): # for reducing the ringing in the sound
    truncated_index = int(np.floor(index))
    next_index = (truncated_index + 1) % wave_table.shape[0]
    next_index_weight = index - truncated_index
    truncated_index_weight = 1 - next_index_weight
    return truncated_index_weight * wave_table[truncated_index] + next_index_weight * wave_table[next_index]

def main():
    sample_rate_samples_per_second = 44100
    waveform_freq_hz = 440
    time_sec = 3
    waveform_type = np.sin
    wavetable_length = 64
    wave_table = np.zeros((wavetable_length,))

    for n in range(wavetable_length):
        wave_table[n] = waveform_type(2*np.pi*n / wavetable_length)

    output = np.zeros((time_sec * sample_rate_samples_per_second,))

    index = 0
    index_increment = waveform_freq_hz * wavetable_length / sample_rate_samples_per_second

    for n in range(output.shape[0]):
        output[n] = interpolate_linearly(wave_table, index)
        index += index_increment
        index %= wavetable_length

    gain = -20
    amplitude = 10 ** (gain / 20)
    output *= amplitude

    wav.write("sine440Hz_scaled_interpolated_linearly.wav", sample_rate_samples_per_second, output.astype(np.float32))

if __name__ == "__main__":
    main()

### Fading at the start and end of the sound

In [4]:
import numpy as np
import scipy.io.wavfile as wav

def interpolate_linearly(wave_table, index): # for reducing the ringing in the sound
    truncated_index = int(np.floor(index))
    next_index = (truncated_index + 1) % wave_table.shape[0]
    next_index_weight = index - truncated_index
    truncated_index_weight = 1 - next_index_weight
    return truncated_index_weight * wave_table[truncated_index] + next_index_weight * wave_table[next_index]

def fade_in_out(signal, fade_length = 1000): # for adding fade at the start and end of the sound 
    fade_in = (1 - np.cos(np.linspace(0, np.pi, fade_length))) * 0.5
    fade_out = np.flip(fade_in)
    signal[:fade_length] = np.multiply(fade_in, signal[:fade_length])
    signal[-fade_length:] = np.multiply(fade_out, signal[-fade_length:])
    return signal

def main():
    sample_rate_samples_per_second = 44100
    waveform_freq_hz = 440
    time_sec = 3
    waveform_type = np.sin
    wavetable_length = 64
    wave_table = np.zeros((wavetable_length,))

    for n in range(wavetable_length):
        wave_table[n] = waveform_type(2*np.pi*n / wavetable_length)

    output = np.zeros((time_sec * sample_rate_samples_per_second,))

    index = 0
    index_increment = waveform_freq_hz * wavetable_length / sample_rate_samples_per_second

    for n in range(output.shape[0]):
        output[n] = interpolate_linearly(wave_table, index)
        index += index_increment
        index %= wavetable_length

    gain = -20
    amplitude = 10 ** (gain / 20)
    output *= amplitude

    output = fade_in_out(output)

    wav.write("sine440Hz_scaled_interpolated_linearly_faded.wav", sample_rate_samples_per_second, output.astype(np.float32))

if __name__ == "__main__":
    main()

### Sawtooth

In [5]:
import numpy as np
import scipy.io.wavfile as wav

def interpolate_linearly(wave_table, index): # for reducing the ringing in the sound
    truncated_index = int(np.floor(index))
    next_index = (truncated_index + 1) % wave_table.shape[0]
    next_index_weight = index - truncated_index
    truncated_index_weight = 1 - next_index_weight
    return truncated_index_weight * wave_table[truncated_index] + next_index_weight * wave_table[next_index]

def fade_in_out(signal, fade_length = 1000): # for adding fade at the start and end of the sound 
    fade_in = (1 - np.cos(np.linspace(0, np.pi, fade_length))) * 0.5
    fade_out = np.flip(fade_in)
    signal[:fade_length] = np.multiply(fade_in, signal[:fade_length])
    signal[-fade_length:] = np.multiply(fade_out, signal[-fade_length:])
    return signal

def sawtooth(x):
    return (x + np.pi) / np.pi % 2 - 1

def main():
    sample_rate_samples_per_second = 44100
    waveform_freq_hz = 220
    time_sec = 3
    waveform_type = sawtooth
    wavetable_length = 64
    wave_table = np.zeros((wavetable_length,))

    for n in range(wavetable_length):
        wave_table[n] = waveform_type(2*np.pi*n / wavetable_length)

    output = np.zeros((time_sec * sample_rate_samples_per_second,))

    index = 0
    index_increment = waveform_freq_hz * wavetable_length / sample_rate_samples_per_second

    for n in range(output.shape[0]):
        output[n] = interpolate_linearly(wave_table, index)
        index += index_increment
        index %= wavetable_length

    gain = -20
    amplitude = 10 ** (gain / 20)
    output *= amplitude

    output = fade_in_out(output)

    wav.write("sawtooth220Hz_scaled_interpolated_linearly_faded.wav", sample_rate_samples_per_second, output.astype(np.float32))

if __name__ == "__main__":
    main()